<a href="https://colab.research.google.com/github/kevinMGII/Grupo7-Practica1/blob/main/Grupo7_Practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PRIMERA PRÁCTICA**.
Predicción del Abandono de Empleados

 ---

Carga y visualización de los datos desde Google Colab.

In [ ]:
import pandas as pd

data_train = pd.read_csv("./attrition_availabledata_12.csv.gz")                 # Cargamos el conjunto de datos de entrenamiento
data_test = pd.read_csv("./attrition_competition_12.csv.gz")                    # Cargamos el conjunto de datos de test utilizando el almacenamiento local de Google Colab.

data_train.head()                                                               # Mostramos las 5 primeras filas del conjunto de datos de entrenamiento. (Not Working)
data_test.head()                                                                # Mostramos las 5 primeras filas del conjunto de datos de test.

En el desarrollo de esta práctica se utilizarán los archivos que contienen los datos con la extension **12**. Cumpliendo así las consideraciones generales, siendo la suma de los últimos dos dígitos nuestros NIA's 12 en ambos casos.

 ---

### **2. EDA SIMPLIFICADO**

A continuación, se realiza un Análisis Exploratorio de Datos (EDA). El objetivo es obtener una visión general del estado de los datos, permitiendo llevar a cabo algunos ajustes o limpiezas antes de empezar a utilizarlos.



**Número de  variables e instancias:**

In [ ]:
num_instancias, num_variables = data_train.shape                                # Usamos el atributo .shape de pandas para obtener la tupla (número de filas, número de columnas)
print(f"Número de instancias: {num_instancias}")                                # Imprimimos num_instancias (número de filas)
print(f"Número de variables: {num_variables}")                                  # Imprimimos num_variables (número de columnas)

El conjunto de datos con el que vamos a llevar a cabo esta práctica esta formado por 2940 instancias y 31 variables. Disponemos de una cantidad suficiente de datos como para entrenar un modelo de forma correcta.

**Tipos de variables (categóricas, numéricas u ordinales):**

In [ ]:
print(data_train.dtypes)                                                              # Mostramos los tipos de datos de cada columna del conjunto de datos de entrenamiento.

                                                                                      # Seleccionamos los tipos de datos categóricos y numéricos con pandas
categorical_vars = data_train.select_dtypes(include=['object', 'category']).columns   # Variables categóricas: aquellas de tipo 'object' o 'category'
numeric_vars = data_train.select_dtypes(include=['int64', 'float64']).columns         # Variables numéricas: aquellas de tipo 'int64' o 'float64'

print(f"Variables categóricas: {categorical_vars}")                                   # Imprimimos las variables categóricas
print(f"Variables numéricas: {numeric_vars}")                                         # Imprimimos las variables numéricas

# Las variables ordinales deben identificarse manualmente (mirar Nota)

*Nota: pandas no es capaz de identificar variables ordinales automáticamente.*


*Debemos analizar cada variable categórica y estudiar si existe un orden jerárquico entre los valores.*


```
# Ver valores únicos de la columna 'X' en el conjunto de entrenamiento
print(data_train['X'].unique())
```


* BusinessTravel (Non-Travel, Travel_Rarely, Travel_Frequently) -> No
* Department: (Sales, Research & Development, ...) -> No
* EducationField (Marketing, Medical, Life Sciences, ...) -> No
* Gender (Male, Female) -> No
* JobRole (Research Director, Sales Executive, Laboratory Technician, ...) -> No
* MaritalStatus (Single, Married, Divorced) -> No
* Over18 (N, Y) -> No
* Attrition (No, Yes) -> No

*Concluimos que no hay variables ordinales.*

**Variables categóricas con alta cardinalidad:**

In [ ]:
for col in data_train.columns:                                                  # Recorremos las columnas del conjunto de datos: data_train
    tipo_columna = data_train[col].dtype                                        # Almacenamos en una variable el tipo de dato de la columna actual
    if tipo_columna == 'object' or tipo_columna.name == 'category':             # Check: ¿El tipo es un objeto, texto?, ¿El tipo es una categoría?
        valores_unicos = data_train[col].unique()                               # Almacenamos en una variable los valores únicos de esa categoría
        num_valores_unicos = len(valores_unicos)                                # Obtenemos la cantidad de valores únicos que hay
        print("Cardinalidad de", col, ":", num_valores_unicos)                  # Mostramos la cardinalidad y el número de valores únicos por pantalla

El resultado obtenido nos indica que la mayoría de variables categóricas en el conjunto de datos presentan una baja cardinalidad. Sin embargo, algunas como "JobRole" tienen una alta cardinalidad con hasta 9 valores distintos.

Además, se puede observar como "Over18" solamente tiene un valor, por lo que no aporta información relevante y debe ser eliminada.

**Variables con valores faltantes y cuántos son:**

In [ ]:
for col in data_train.columns:                                                  # Recorremos las columnas del conjunto de datos: data_train
    contador_faltantes = 0                                                      # Inicializamos el contador de faltantes a 0
    for valor in data_train[col]:                                               # Recorremos los valores de la columna
        if valor is None or valor != valor:                                     # Check: ¿El valor es nulo (faltante)? Especial: valor != valor para detectar NaN
            contador_faltantes += 1                                             # Incrementamos el contador de faltantes en una unidad
    if contador_faltantes > 0:                                                  # Check: ¿Hay valores faltantes en esa columna?
        print("Variable:", col, "- Valores faltantes:", contador_faltantes)     # Mostramos la variable con sus faltantes por pantalla

Tras la ejecución, se puede observar como hay cinco variables presentes en el modelo que contienen valores faltantes, destacando alguna como "WorkLifeBalance" con hasta 23 valores no presentes, o "EnvironmentSatisfaction" con hasta 17.

Durante el desarrollo de la práctica determinaremos que estrategia seguir para solventar este problema, analizando el impacto de estas variables.

**Columnas constantes o columnas de ID:**

In [ ]:
for col in data_train.columns:                                                  # Recorremos las columnas del conjunto de datos: data_train
    valores_unicos = data_train[col].unique()                                   # Obtenemos los valores únicos de la columna
    if len(valores_unicos) == 1 and valores_unicos[0] == valores_unicos[0]:     # Check: ¿La columna tiene un solo valor? Especial: ...[0] == ...[0] para detectar NaN
        print("Variable:", col, "- Columna constante")                          # Mostramos la variable con la columna constante
    elif len(valores_unicos) == len(data_train):                                # Check: ¿El número de valores únicos es igual al número de filas?
        print("Posible columna de ID:", col)                                    # Mostramos la posible variable o columna de ID por pantalla

Se identifican tres columnas constantes: "EmployeeCount", "Over18" y "StandardHours", así como una posible columna de id: "EmployeeID".

Las columnas constantes no aportan información relevante o que resulte útil al modelo, y pueden eliminarse.

La columna de id contiene valores únicos para cada fila, por lo que no contribuye al modelo y puede eliminarse.

**Comprobación del tipo de problema: ¿clasificación o regresión?**

In [ ]:
datos_col = data_train["Attrition"]                                             # Seleccionamos la columna "Attrition" del conjunto de datos de entrenamiento
valores_unicos = datos_col.unique()                                             # Obtenemos los valores únicos de la columna "Attrition"
print(valores_unicos)                                                           # Mostramos los valores únicos por pantalla

Como se puede observar en el resultado de la ejecución, se trata de un problema de clasificación binaria. El modelo podrá predecir una de las dos categorías: No, Yes.

**Puesto que es un problema de clasificación, ¿está desbalanceado?**

In [ ]:
datos_col = data_train["Attrition"]                                             # Seleccionamos la columna "Attrition" del conjunto de datos de entrenamiento
contador_clases = datos_col.value_counts()                                      # Contamos la cantidad de valores de cada clase en la columna "Attrition"
print(contador_clases)                                                          # Mostramos la cantidad de valores de cada clase por pantalla

Concluimos el Análisis Exploratorio de Datos (EDA) indicando que el problema de clasificación está desbalanceado. Esto es debido a que existe una mayor cantidad de valores en una categoría: "No", respecto a la otra: "Yes".

Un conjunto de datos desbalanceado puede provocar que el modelo aprenda mejor la categoría con mayor cantidad de valores y no aprenda correctamente la categoría con menor cantidad de valores.



---

